In [1]:
## Load all libraries and setup the synthetic problem
from library.Mag import Mag, ProblemSetter, MathUtils, Simulator, DataIO
import numpy as np
from SimPEG import PF, Utils, Mesh, Maps
from SimPEG import Utils
from SimPEG.Utils import mkvc
import SimPEG.PF as PF
import scipy as sp
import re
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt
import ipywidgets as widgets

# Load data and topo and build default model
fName = 'DataGrid.tiff'

dataGrid = DataIO.loadGeoTiffFile(fName, plotIt=True)

<Figure size 800x800 with 2 Axes>

# 2.1.2.	2D Magnetic Data Filters#


2D magnetic data filters are commonly applied to magnetic data to allow us to view that data in different ways. They can highlight shallower or deeper magnetic sources, and can emphasize gradients in the magnetic data that occur at geological contacts and in association with structure.

We use the same synthetic model here as in the previous notebook to demonstrate the effects of various 2D magnetic filters.



## Total magnetic intensity data

The below image show the total magnetic intensity (TMI) that was calculated from the synthetic geologic model. The response is based on magnetic field parameters at the north pole, in other words, the magnetic field is vertical.  

## X and Y derivatives of the magnetic response

Calculating horizontal and vertical derivatives of the magnetic response gives us a way to better visualize local gradients in the data that can be obscured by larger and deeper features in the total field response. X and Y derivatives are calculated based on the difference observed in magnetic response between adjacent points, or grid cells, in the X and Y directions, respectively. Steep gradients usually occur at geologic contacts, faults, or fractures, where there is a distinct contrast in magnetic susceptibility. Interpreters should not consider X and Y derivative results independently, and should be sure to evaluate both, since visualizing gradients in one direction will de-emphasize features trending in the perpendicular direction. The X and Y derivatives are also used in the calculation of several other magnetic data filters.

Grids showing the X and Y derivatives are shown below. Note the emphases on east and west geological boundaries using the X derivative, and on the north and south geological boundaries using the Y derivative (best seen in relation to the smaller magnetic anomaly). Image sun-shading and color can be varied, and the filtered data can be compared with the original total mangetic intensity (TMI) data using the 'Grid Filters' drop-down menu. 

In [2]:
survey = dataGrid
xLoc = np.asarray(range(survey.nx))*survey.dx+survey.limits[0]
yLoc = np.asarray(range(survey.ny))*survey.dy+survey.limits[2]
xlim = survey.limits[:2]
ylim = survey.limits[2:]

In [3]:
param = Simulator.gridFiltersWidget(dataGrid, gridFilter='derivativeX', EPSGCode=26909)
display(param)

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='SunAzimuth', max=360.0, ste…

In [4]:
param = Simulator.gridFiltersWidget(dataGrid, gridFilter='derivativeY', EPSGCode=26909)
display(param)

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='SunAzimuth', max=360.0, ste…

## Upward continuation

Upward continuation can be considered a ‘wavelength’ filter. Upward continuation simulates the magnetic response that would be observed if data were collected at a greater height above the Earth’s surface than it was actually collected. The result is that longer-wavelength, deeper features are emphasized over shorter-wavelength, near-surface features. It is effective for interpretation of deeper geology, or for reducing noise that may be found in data collected very close to the ground. 

You can alternate between the upward contined response here and the TMI data. Change the upward continuation distance using the slider bar. Note the small magnetic anomaly and the narrow fault feature become less distinct with increased distance. 

In [5]:
param = Simulator.gridFiltersWidget(dataGrid, gridFilter='upwardContinuation', EPSGCode=26909)
display(param)

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='SunAzimuth', max=360.0, ste…

## Vertical derivative

A very commonly used magnetic filter is the first vertical derivative. This process considers the change in the magnetic response when calculated at two different heights above the ground. The effect is to emphasize near-surface features where the difference between the responses calculated at two different heights will be largest. First vertical derivative anomalies will highlight edges of magnetic sources, and appear over the top of the source if the contact or feature is vertical. 

Notice how the first vertical derivative peaks directly over the small magnetic body from the synthetic geologic model.

In [6]:
param = Simulator.gridFiltersWidget(dataGrid, gridFilter='firstVertical', EPSGCode=26909)
display(param)

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='SunAzimuth', max=360.0, ste…

## Total horizontal derivative

The total horizontal derivative simply combines the X and Y derivatives. Gradients in both directions are now accounted for in a single map. The highest total horizontal derivative values occur at the edges or boundaries of magnetic sources. It does not detect narrow sources as effectively as the vertical derivative.

The total horizonal derivative can be seen to peak here over the edges of the small magnetic body, and along the margin of the large magnetic block in the east. 

In [7]:
param = Simulator.gridFiltersWidget(dataGrid, gridFilter='totalHorizontal', EPSGCode=26909)
display(param)

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='SunAzimuth', max=360.0, ste…

## Tilt angle

The tilt derivative or tilt angle normalizes the vertical derivative by the horizontal derivatives. A magnetic source exhibiting a strong contrast with surrounding rocks (e.g. a large, near surface, magnetic unit) will yield both high vertical and horizontal gradients, and a more weakly contrasting body will yield proportionally smaller vertical and horizontal gradients. Normalizing the vertical by horizontal derivatives means different amplitude responses are assigned equivalent values. This is a very useful filter for enhancing more subtle features in the magnetic data. The tilt angle ranges from -90 to +90 degrees, is positive over the magnetic source, and negative outside the source, with the body’s edge delimited by the 0 degree contour.

It is helpful to view the tilt angle using a color map that highlights the middle, or near-zero tilt angle values, which should trace source edges. For example, the red-blue color map (RdBu). 

In the grid image below, you can see that zero values trace the edge of the small magnetic body, with positive tilt angle values directly over the top of the body (inside) and negative values outside the magnetic body. Zero values also mark the edge of the large eastern magnetic block, as well as the margins of the northwest-trending fault that cross-cuts the eastern block.   

In [8]:
param = Simulator.gridFiltersWidget(dataGrid, gridFilter='tiltAngle', EPSGCode=26909)
display(param)

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='SunAzimuth', max=360.0, ste…

## Analytic signal

Analytic signal is also known as the total gradient. It is calculated from the vertical and horizontal derivatives. The advantage is that we now capture gradients in all three directions, further enhancing detection of geological boundaries and structures. The analytic signal peaks above narrow bodies and along the edges of larger geologic features that are in magnetic contrast to their surroundings. The analytic signal is a very useful interpretation product in areas where magnetic remanence is suspected, and in areas of low latitude since it is not affected by magnetisation direction.  

The analytic signal from the synthetic model looks similar to the first vertical derivative, with the strongest anomalies over the small magnetic body and the edge of the eastern magnetic block.    

In [9]:
param = Simulator.gridFiltersWidget(dataGrid, gridFilter='analyticSignal', EPSGCode=26909)
display(param)

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='SunAzimuth', max=360.0, ste…